In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl

dataset of CVRs

https://dataverse.harvard.edu/dataverse/rcv_cvrs

In [9]:
class VotingData():
    """
    The PrefList class is an object that will hold all the data of a certain election.
    """
    def __init__(self, filepath, num_ranks):
        self.filepath = filepath
        self.num_ranks = num_ranks
        
        ranks = []
        for i in range(1, num_ranks + 1):
            ranks.append('rank'+str(i))

        df = pd.read_csv(filepath, usecols=ranks)
        print("read csv")

        def _left_adj_and_remove_dupes(row, n=5):
            """
            Shifts values in a row of n columns to the left so all '' appear to the right.
            """
            seen = []
            for i in range(n):
                if row.iloc[i] in seen:
                    row.iloc[i] = ''
                if row.iloc[i] == '' and i < n - 1:
                    j = i + 1
                    swapped = False
                    while j <= n-1 and not swapped:
                        if row.iloc[j] in seen:
                            row.iloc[j] = ''
                        if row.iloc[j] != '':
                            row.iloc[i], row.iloc[j] = row.iloc[j], row.iloc[i]
                            swapped = True
                        j += 1
                    if not swapped:
                        return row
                seen.append(row.iloc[i])
            return row

        df = df.groupby(ranks, as_index=False).size()
        df = df.replace(['skipped','overvote'], '')
        print("replaced null votes")
        df = df.groupby(ranks, as_index=False).sum()
        df = df.apply(_left_adj_and_remove_dupes, axis = 1, args=(num_ranks,))
        df = df.groupby(ranks, as_index=False).sum()
        print("left adjusted")
        df = df.loc[df['rank1'] != '']
        df = df.rename(columns={'size':'count'})
        
        self.ballots = df.sort_values('count', ascending=False)
        self.candidates = [x for x in pd.unique(df[ranks].to_numpy().ravel()) if x != '']
        print("done init")
        
        def _remove_and_left_adj(row, n, removed):
            """
            input: row is a row in a ballot. It is assumed that all none '' entries come first and are unique.
                n is the number of ranks in the ballot. 
                removed is the candidate to be removed

            output: a row with removed replaced by '' and all entries left adjusted.
            """
            i = 0
            found = False
            while i < n:
                if row[i] == removed:
                    found = True
                    break
                i += 1

            if found:
                while i < n and row[i] != '':
                    if i == n - 1:
                        row[i] = ''
                    else:
                        row[i] = row[i+1]
                    i += 1
            return row
    
    def plurality_winner(self):
        df = self.ballots[['rank1','count']].groupby('rank1').sum().sort_values('count',ascending=False)
        num_votes = df['count'].sum()
        df['percent'] = round(100*df['count']/num_votes, 1)
        return df
    
    def instant_runoff_winner(self):
            df = self.ballots
            winner =  False
            result = pd.DataFrame({}, index=self.candidates) 
            rnd = 1
            while True:
                curr_rnd = df[['rank1','count']].groupby('rank1').sum().sort_values('count',ascending=False)
                num_votes = curr_rnd['count'].sum()
                curr_rnd['percent'] = round(100*curr_rnd['count']/num_votes, 1)
                curr_rnd = curr_rnd.rename(columns={'count':'count '+str(rnd),'percent':'percent '+str(rnd)})

                # append curr_rnd to results
                result = pd.concat([result,curr_rnd], axis=1)

                if curr_rnd.max()['percent '+str(rnd)] > 50:
                    result = result.sort_values('count 1', ascending=False)
                    return result
                else:
                    loser = curr_rnd.idxmin()['count '+str(rnd)]
                    df = df.apply(_remove_and_left_adj, axis=1, args=(self.num_ranks,loser))
                    df = df.groupby(ranks, as_index=False).sum()
                    df = df.loc[df['rank1'] != '']
                rnd += 1
    
    def STV_winners(self, n):
        """
        input: n is the number of candidates being elected.
        
        output: The results of the single transferable vote algorithm.
        """
        pass
        
    def pairwise_winners(self):
        """
        returns a matrix showing the one on one results of row versus column.
        """
        res = pd.DataFrame(0,columns = self.candidates, index=self.candidates)
        
        def _count_pairwise_scores(row, n):
            for i in range(n-1):
                if row.iloc[i] == '':
                    break
                for j in range(i+1,n):
                    if row.iloc[j] == '':
                        break
                    res.loc[row[i],row[j]] += row['count']

        self.ballots.apply(_count_pairwise_scores, axis=1, args = (self.num_ranks,))
        res = round(100*res/(res + res.T),1)
#         res.style.highlight_between(left=50.1, right=100, color="#33db27")
        return res
        

In [10]:
%%time
ny_voting_data = VotingData("NewYorkCity_06222021_DEMMayorCitywide.csv", 5)

read csv
replaced null votes


KeyboardInterrupt: 

In [4]:
%%time
ny_voting_data.pairwise_winners()

NameError: name 'ny_voting_data' is not defined

In [61]:
%%time
# import matplotlib as mpl

balls = ny_voting_data.ballots
cands = ny_voting_data.candidates
res = pd.DataFrame(0,columns = cands, index=cands)
votes = pd.DataFrame(0,columns = cands, index=cands)


def _count_pairwise_scores(row, n):
    for i in range(n-1):
        if row[i] == '':
            break
        for j in range(i+1,n):
            if row[j] == '':
                break
            res.loc[row[i],row[j]] += row['count']
    
balls.apply(_count_pairwise_scores, axis=1, args = (5,))
res = round(100*res/(res + res.T),1)

res.style.highlight_between(left=50.1, right=100, color="#19b543")
# res = res.background_gradient(cmap=cm)

CPU times: user 1min 4s, sys: 83.9 ms, total: 1min 4s
Wall time: 1min 19s


,Aaron S. Foldenauer,Andrew Yang,Art Chang,Dianne Morales,Joycelyn Taylor,Kathryn A. Garcia,Scott M. Stringer,Shaun Donovan,Isaac Wright Jr.,writein,Eric L. Adams,Maya D. Wiley,Raymond J. McGuire,Paperboy Love Prince
Aaron S. Foldenauer,nan,24.900000,43.200000,46.200000,53.600000,31.000000,40.500000,34.400000,56.700000,63.300000,23.900000,35.100000,49.600000,58.700000
Andrew Yang,75.100000,nan,70.700000,58.700000,79.400000,40.400000,54.500000,52.900000,65.800000,82.900000,34.100000,36.800000,57.700000,74.400000
Art Chang,56.800000,29.300000,nan,40.000000,67.500000,35.100000,44.200000,52.100000,67.000000,72.900000,32.500000,17.700000,44.100000,63.900000
Dianne Morales,53.800000,41.300000,60.000000,nan,72.200000,37.700000,46.500000,53.000000,65.900000,79.000000,28.300000,18.300000,49.100000,75.200000
Joycelyn Taylor,46.400000,20.600000,32.500000,27.800000,nan,22.300000,31.000000,29.200000,45.900000,69.800000,10.800000,15.100000,32.000000,52.000000
Kathryn A. Garcia,69.000000,59.600000,64.900000,62.300000,77.700000,nan,68.700000,77.100000,75.200000,83.800000,49.100000,44.700000,71.300000,78.500000
Scott M. Stringer,59.500000,45.500000,55.800000,53.500000,69.000000,31.300000,nan,58.700000,68.600000,77.500000,30.400000,30.700000,53.000000,73.500000
Shaun Donovan,65.600000,47.100000,47.900000,47.000000,70.800000,22.900000,41.300000,nan,58.100000,80.000000,24.000000,26.700000,47.700000,69.000000
Isaac Wright Jr.,43.300000,34.200000,33.000000,34.100000,54.100000,24.800000,31.400000,41.900000,nan,64.800000,10.600000,18.400000,30.100000,50.700000
writein,36.700000,17.100000,27.100000,21.000000,30.200000,16.200000,22.500000,20.000000,35.200000,nan,13.700000,13.000000,20.200000,30.600000
